In [4]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime as dt
import os

In [2]:
def kelvin2celsius(k:float) -> float:
    return k-273.15

def nan_floating(i:float) -> float or np.nan:
    if abs(i) < 0.0005:
        return np.nan
    else:
        return i

In [3]:
date = dt.datetime(2022,1,1)
ds = xr.open_dataset(f"../data/{date.year}/{str(date.month).zfill(2)}/{date.year}_{str(date.month).zfill(2)}{str(date.day).zfill(2)}.nc")
ds

<xarray.Dataset>
Dimensions:     (lon: 481, lat: 505, time: 24, ref_time: 8)
Coordinates:
  * lon         (lon) float32 120.0 120.1 120.1 120.2 ... 149.9 149.9 150.0
  * lat         (lat) float32 47.6 47.55 47.5 47.45 ... 22.55 22.5 22.45 22.4
  * time        (time) datetime64[ns] 2022-01-01 ... 2022-01-01T23:00:00
  * ref_time    (ref_time) datetime64[ns] 2022-01-01 ... 2022-01-01T21:00:00
Data variables:
    psea        (time, lat, lon) float64 ...
    sp          (time, lat, lon) float64 ...
    u           (time, lat, lon) float64 ...
    v           (time, lat, lon) float64 ...
    temp        (time, lat, lon) float64 ...
    rh          (time, lat, lon) float64 ...
    r1h         (time, lat, lon) float64 ...
    ncld_upper  (time, lat, lon) float64 ...
    ncld_mid    (time, lat, lon) float64 ...
    ncld_low    (time, lat, lon) float64 ...
    ncld        (time, lat, lon) float64 ...
    dswrf       (time, lat, lon) float64 ...
Attributes:
    Conventions:  CF-1.0
    history:      created by create_1daync_msm_s.rb  2022-01-03

In [4]:
tokyo_ds = ds.sel(lat=35,lon=139,method="nearest")
tokyo_temp = tokyo_ds.temp
tokyo_zero = tokyo_temp.values
pd.Series(tokyo_ds.temp.values).apply(kelvin2celsius)
# kelvin2celsius(tokyo_zero)

0     1.569426
1     3.032981
2     3.947703
3     4.731750
4     5.552387
5     6.145649
6     6.043723
7     5.113320
8     3.430232
9     1.658284
10    1.342052
11    1.315917
12    0.257453
13   -0.228656
14   -0.006510
15   -0.769649
16   -1.587672
17   -0.126730
18    0.181662
19   -0.150252
20    0.628569
21   -0.192068
22    0.071895
23    2.131326
dtype: float64

In [5]:
date

datetime.datetime(2022, 1, 1, 0, 0)

In [6]:
cols = ["date(UTC)","海面気圧","地上気圧","東風","西風","気温","相対湿度","降水量","雲量","下行短波放射線フラックス"]
df = pd.DataFrame(columns=cols)
df["気温"] = pd.Series(tokyo_zero).apply(kelvin2celsius)
df["地上気圧"] = pd.Series(tokyo_ds.sp.values).apply(kelvin2celsius)
df["date(UTC)"] = pd.date_range(start=date,periods=24,freq="h")
df["相対湿度"] = pd.Series(tokyo_ds.rh.values)
df["降水量"] = pd.Series(tokyo_ds.r1h.values)
df["雲量"] = pd.Series(tokyo_ds.ncld.values)
df["海面気圧"] = pd.Series(tokyo_ds.psea.values)
df["東風"] = pd.Series(tokyo_ds.u.values)
df["西風"] = pd.Series(tokyo_ds.v.values)
df["下行短波放射線フラックス"] = pd.Series(tokyo_ds.dswrf.values)
df

,date(UTC),海面気圧,地上気圧,東風,西風,気温,相対湿度,降水量,雲量,下行短波放射線フラックス
0,2022-01-01 00:00:00,101969.265927,98403.913877,1.039755,0.000000,1.569426,44.800460,200.000000,0.000002,665.0000
1,2022-01-01 01:00:00,101951.376019,98404.831308,2.012232,-0.269113,3.032981,38.495414,-0.000007,0.000002,405.6340
2,2022-01-01 02:00:00,101982.109963,98436.941399,2.287462,-0.226300,3.947703,30.208718,-0.000007,0.000002,517.7485
3,2022-01-01 03:00:00,101870.183360,98347.950575,2.446483,0.287462,4.731750,32.116974,200.000000,0.000002,665.0000
4,2022-01-01 04:00:00,101800.917306,98279.143237,2.195719,0.152905,5.552387,27.905965,-0.000007,0.000002,555.6735
5,2022-01-01 05:00:00,101869.724644,98363.546905,1.938838,-0.030581,6.145649,25.795874,-0.000007,0.000002,478.8600
6,2022-01-01 06:00:00,101930.733818,98418.592776,1.688073,0.972477,6.043723,32.371561,200.000000,0.000002,665.0000
7,2022-01-01 07:00:00,102027.522807,98509.418462,-0.183486,0.366972,5.113320,45.087157,-0.000007,0.000002,171.4625
8,2022-01-01 08:00:00,102083.486108,98555.290021,-1.516820,-0.660550,3.430232,42.883029,-0.000007,0.000002,22.5095
9,2022-01-01 09:00:00,102217.889776,98672.721211,-1.327217,-0.366972,1.658284,46.639909,200.000000,0.000002,665.0000


In [1]:
import sys
sys.path.append("../modules/")

import netCDF2csv as ncv

In [2]:
cd = ncv.CreateDataSet(2022,1,1)
cd.create_dataframe()

In [8]:
path = "C:/Users/astro/Documents/Python_Project/ForecastCloudinJapan/dev/netCDF2csv.py"
"/".join(path.split("/")[:-1])

'C:/Users/astro/Documents/Python_Project/ForecastCloudinJapan/dev'